In [ ]:
import polars as pl
import polars.selectors as cs

# Create `anzsic_1993.csv`

In [ ]:
anzsic93_raw = (
    pl.read_excel(
        r"data/1292.0.55.005_ anzsic 2006 - correspondence tables.xls",
        sheet_name="Table 1",
        drop_empty_rows=True,
    )
    .with_row_index()
    .filter(~pl.col("index").is_between(0, 2))
    .drop(~cs.by_index(0, 1, 2, 3))
)

anzsic93_raw.columns = ["index", "parent", "code", "title"]

In [ ]:
def tidy_anzsic93(df, name, code_len):
    df_final = df.filter(pl.col("parent").str.len_chars() == code_len).drop(
        cs.by_index(-1)
    )
    df_final.columns = ["index", "code", "title"]
    df_final = df_final.sort("index").rename(
        lambda colname: name[:-1].lower() + "_" + colname
    )
    return df_final


anzsic93 = {}

for index, name in enumerate(["Divisions", "Subdivisions", "Groups"]):
    anzsic93[name] = tidy_anzsic93(anzsic93_raw, name, index + 1)

In [ ]:
# We can infer the relationship between levels using their row index.
anzsic93_combined = (
    (
        anzsic93_raw.filter(~pl.col("title").is_null())
        .drop("parent")
        .rename(lambda col: "class_" + col)
    )
    .join_asof(
        anzsic93["Groups"],
        left_on="class_index",
        right_on="group_index",
        strategy="backward",
    )
    .join_asof(
        anzsic93["Subdivisions"],
        left_on="group_index",
        right_on="subdivision_index",
        strategy="backward",
    )
    .join_asof(
        anzsic93["Divisions"],
        left_on="subdivision_index",
        right_on="division_index",
        strategy="backward",
    )
    .drop(cs.ends_with("_index"))
    .select(
        cs.starts_with("division"),
        cs.starts_with("subdivision"),
        cs.starts_with("group"),
        cs.starts_with("class"),
    )
)

In [ ]:
anzsic93_combined.write_csv(r"anzsic_1993.csv", quote_style="always")

## Create `anzsic_2006.csv`

In [ ]:
def read_anzsic_sheet(sheet_name, prefix):
    df = (
        pl.read_excel(
            r"data/1292.0.55.002_anzsic 2006 - codes and titles.xls",
            sheet_name=sheet_name,
            drop_empty_rows=True,
        )
        .drop(cs.by_index(0))
        .select(cs.by_index(-2, -1))
        .with_columns(pl.all().fill_null(strategy="forward"))
        .drop_nulls()
    )
    df.columns = [f"{prefix}_code", f"{prefix}_title"]
    df = df.unique(f"{prefix}_title", keep="first", maintain_order=True).sort(
        f"{prefix}_code"
    )
    if prefix != "division":
        df = df.with_columns(
            pl.col(f"{prefix}_code").str.replace(r".$", "").alias(f"{prefix}_parent")
        )
    return df


anzsic06 = {
    sheet_name: read_anzsic_sheet(sheet_name, prefix)
    for sheet_name, prefix in zip(
        ["Divisions", "Groups", "Classes"], ["division", "group", "class"]
    )
}

anzsic06

In [ ]:
# Create Subdivisions
anzsic06["Subdivisions"] = pl.read_excel(
    r"data/1292.0.55.002_anzsic 2006 - codes and titles.xls",
    sheet_name="Subdivisions",
    drop_empty_rows=True,
).drop(cs.by_index(0))

anzsic06["Subdivisions"] = (
    anzsic06["Subdivisions"]
    .with_columns(pl.all().fill_null(strategy="forward"))
    .drop_nulls()
)

anzsic06["Subdivisions"].columns = [
    "division_code",
    "subdivision_code",
    "subdivision_title",
]

anzsic06["Subdivisions"] = (
    anzsic06["Subdivisions"]
    .unique("subdivision_title")
    .sort(["division_code", "subdivision_code"])
)

anzsic06["Subdivisions"]

In [ ]:
anzsic06_combined = (
    anzsic06["Divisions"]
    .join(anzsic06["Subdivisions"], on="division_code", how="full")
    .join(
        anzsic06["Groups"],
        left_on="subdivision_code",
        right_on="group_parent",
        how="full",
    )
    .join(
        anzsic06["Classes"], left_on="group_code", right_on="class_parent", how="full"
    )
    .drop(cs.ends_with("parent"), cs.ends_with("_right"))
)

In [ ]:
anzsic06_combined.write_csv("anzsic_2006.csv", quote_style="always")

# Create `anzsic_1993_to_2006_conversion_table.csv`

In [ ]:
anzsic_mapping = (
    pl.read_excel(
        r"data/1292.0.55.005_ anzsic 2006 - correspondence tables.xls",
        sheet_name="Table 1",
        drop_empty_rows=True,
    )
    .with_row_index()
    .filter(~pl.col("index").is_between(0, 2))
    # .drop(~cs.by_index(0, 1, 2, 3))
)
anzsic_mapping.columns = [
    "index",
    "anzsic93_parent",
    "anzsic93_code",
    "anzsic93_title",
    "anzsic06_code",
    "anzsic06_title",
]

anzsic_mapping

In [ ]:
anzsic_1993_classes = anzsic_mapping.filter(
    pl.col("anzsic93_code").str.len_chars() == 4
).select(cs.by_name("index", "anzsic93_code", "anzsic93_title"))
anzsic_1993_classes

In [ ]:
anzsic_2006_classes = anzsic_mapping.filter(
    pl.col("anzsic06_code").is_not_null()
).select("index", cs.starts_with("anzsic06"))
anzsic_2006_classes

In [ ]:
anzsic_ct = (
    anzsic_2006_classes.join_asof(anzsic_1993_classes, on="index")
    .drop("index")
    .select(cs.contains("93"), cs.contains("06"))
)

anzsic_ct.columns = [
    "anzsic_code_93",
    "anzsic_name_93",
    "anzsic_code_06",
    "anzsic_name_06",
]

anzsic_ct.write_csv(r"anzsic_1993_to_2006_conversion_table.csv", quote_style="always")